## Data Loading
import library yang dibutuhkan

In [1]:
#import library
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from google.colab import files
import pandas as pd
import numpy as np

In [2]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"itsself","key":"a32e73a92ced424f8e81358301aa3c3b"}'}

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d jealousleopard/goodreadsbooks

  0% 0.00/622k [00:00<?, ?B/s]
100% 622k/622k [00:00<00:00, 88.7MB/s]


In [5]:
!unzip /content/goodreadsbooks.zip

Archive:  /content/goodreadsbooks.zip
  inflating: books.csv               


mencetak dataset books.csv

In [6]:
books = pd.read_csv("/content/books.csv", error_bad_lines = False)

print("jumlah baris dan kolom pada data buku yang tersedia: ", books.shape)

jumlah baris dan kolom pada data buku yang tersedia:  (11123, 12)


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 3350: expected 12 fields, saw 13\nSkipping line 4704: expected 12 fields, saw 13\nSkipping line 5879: expected 12 fields, saw 13\nSkipping line 8981: expected 12 fields, saw 13\n'


## Univariate Exploratory Data Analysis

In [7]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11123 entries, 0 to 11122
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   bookID              11123 non-null  int64  
 1   title               11123 non-null  object 
 2   authors             11123 non-null  object 
 3   average_rating      11123 non-null  float64
 4   isbn                11123 non-null  object 
 5   isbn13              11123 non-null  int64  
 6   language_code       11123 non-null  object 
 7     num_pages         11123 non-null  int64  
 8   ratings_count       11123 non-null  int64  
 9   text_reviews_count  11123 non-null  int64  
 10  publication_date    11123 non-null  object 
 11  publisher           11123 non-null  object 
dtypes: float64(1), int64(5), object(6)
memory usage: 1.0+ MB


Dataset yang digunakan memiliki 11123 records dan 12 columns:
1. bookID = kode unik buku
2. title = judul buku
3. authors = nama pengarang buku
4. average_rating = penilaian rata-rata yang diberikan pengguna
5. isbn = kode unik standar internasional
6. isbn13 = 13 digit kode unik standar international
7. language_code = bahasa yang digunakan pada buku
8. num_pages = jumlah halaman buku
9. rating_count = jumlah rating
10. text_reviews_count = jumlah ulasan teks pada buku
11. publication_date = tanggal publikasi buku
12. publisher = penerbit dari buku 

menampilkan 5 data teratas pada data set

In [8]:
books.head()

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,0439554896,9780439554893,eng,352,6333,244,11/1/2003,Scholastic
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,5/1/2004,Scholastic Inc.
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,0439682584,9780439682589,eng,2690,41428,164,9/13/2004,Scholastic


melakukan pencetakan jumlah dari judul buku dan judul buku

In [9]:
print("Jumlah judul buku: ", len(books["title"].unique()))
print("Judul buku: ", books["title"].unique())

Jumlah judul buku:  10348
Judul buku:  ['Harry Potter and the Half-Blood Prince (Harry Potter  #6)'
 'Harry Potter and the Order of the Phoenix (Harry Potter  #5)'
 'Harry Potter and the Chamber of Secrets (Harry Potter  #2)' ...
 'The Ice-Shirt (Seven Dreams #1)' 'Poor People'
 'Las aventuras de Tom Sawyer']


## Data Preprocessing

membuat dataframe dengan variabel rating

In [10]:
ratings = books[["isbn", "average_rating", "ratings_count"]]
ratings

,isbn,average_rating,ratings_count
0,0439785960,4.57,2095690
1,0439358078,4.49,2153167
2,0439554896,4.42,6333
3,043965548X,4.56,2339585
4,0439682584,4.78,41428
...,...,...,...
11118,1560254416,4.06,156
11119,0140110879,4.08,783
11120,0140131965,3.96,820
11121,0060878827,3.72,769


menampilkan informasi pada dataframe rating

In [11]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11123 entries, 0 to 11122
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   isbn            11123 non-null  object 
 1   average_rating  11123 non-null  float64
 2   ratings_count   11123 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 260.8+ KB


melihat distribusi rating

In [12]:
ratings.describe()

,average_rating,ratings_count
count,11123.000000,1.112300e+04
mean,3.934075,1.794285e+04
std,0.350485,1.124992e+05
min,0.000000,0.000000e+00
25%,3.770000,1.040000e+02
50%,3.960000,7.450000e+02
75%,4.140000,5.000500e+03
max,5.000000,4.597666e+06


mencetak jumlah data isbn dan jumlah data pada rating

In [13]:
print("jumlah ISBN: ", len(ratings["isbn"].unique()))
print("jumlah data rating: ", len(ratings))

jumlah ISBN:  11123
jumlah data rating:  11123


melakukan pengelompokan dataframe berdasarkan isbn

In [14]:
books.groupby("isbn").sum()

,bookID,average_rating,isbn13,num_pages,ratings_count,text_reviews_count
isbn,,,,,,
000100039X,2547,4.23,9780001000391,127,184293,5418
0001713191,29085,4.17,9780001713192,64,5,0
0002310198,16340,3.87,9780002310192,251,47,4
0002317850,16300,3.95,9780002317856,242,19657,884
0006163831,22868,3.98,9780006163831,479,174,3
...,...,...,...,...,...,...
987224975X,22380,3.97,9789872249755,96,7,2
9875661708,32202,3.70,9789875661707,336,47,2
9875801445,31761,4.32,9789875801448,445,66,4


membuat data baru berisikan data rating 

In [15]:
all_book_rate = ratings
all_book_rate

,isbn,average_rating,ratings_count
0,0439785960,4.57,2095690
1,0439358078,4.49,2153167
2,0439554896,4.42,6333
3,043965548X,4.56,2339585
4,0439682584,4.78,41428
...,...,...,...
11118,1560254416,4.06,156
11119,0140110879,4.08,783
11120,0140131965,3.96,820
11121,0060878827,3.72,769


membuat data baru berisi nama pengarang kemudian digabungkan dengan data all_book_rate 

In [16]:
all_book_author = pd.merge(all_book_rate, books[["isbn", "authors"]], on="isbn", how="left")
all_book_author

,isbn,average_rating,ratings_count,authors
0,0439785960,4.57,2095690,J.K. Rowling/Mary GrandPré
1,0439358078,4.49,2153167,J.K. Rowling/Mary GrandPré
2,0439554896,4.42,6333,J.K. Rowling
3,043965548X,4.56,2339585,J.K. Rowling/Mary GrandPré
4,0439682584,4.78,41428,J.K. Rowling/Mary GrandPré
...,...,...,...,...
11118,1560254416,4.06,156,William T. Vollmann/Larry McCaffery/Michael He...
11119,0140110879,4.08,783,William T. Vollmann
11120,0140131965,3.96,820,William T. Vollmann
11121,0060878827,3.72,769,William T. Vollmann


membuat data baru berisi judul dari buku kemudian digabungkan dengan data all_book_author

In [17]:
all_book = pd.merge(all_book_author, books[["isbn", "title"]], on="isbn", how="left")
all_book

,isbn,average_rating,ratings_count,authors,title
0,0439785960,4.57,2095690,J.K. Rowling/Mary GrandPré,Harry Potter and the Half-Blood Prince (Harry ...
1,0439358078,4.49,2153167,J.K. Rowling/Mary GrandPré,Harry Potter and the Order of the Phoenix (Har...
2,0439554896,4.42,6333,J.K. Rowling,Harry Potter and the Chamber of Secrets (Harry...
3,043965548X,4.56,2339585,J.K. Rowling/Mary GrandPré,Harry Potter and the Prisoner of Azkaban (Harr...
4,0439682584,4.78,41428,J.K. Rowling/Mary GrandPré,Harry Potter Boxed Set Books 1-5 (Harry Potte...
...,...,...,...,...,...
11118,1560254416,4.06,156,William T. Vollmann/Larry McCaffery/Michael He...,Expelled from Eden: A William T. Vollmann Reader
11119,0140110879,4.08,783,William T. Vollmann,You Bright and Risen Angels
11120,0140131965,3.96,820,William T. Vollmann,The Ice-Shirt (Seven Dreams #1)
11121,0060878827,3.72,769,William T. Vollmann,Poor People


## Data Preparation

melakukan pemeriksaan pada data apakah terdapat missing value atau tidak

In [18]:
# Mengecek missing value pada dataframe all_resto
all_book.isnull().sum()

isbn              0
average_rating    0
ratings_count     0
authors           0
title             0
dtype: int64

mengurutkan data berdasarkan isbn

In [19]:
fix_book = all_book.sort_values("isbn", ascending=True)
fix_book

,isbn,average_rating,ratings_count,authors,title
781,000100039X,4.23,184293,Kahlil Gibran/جبران خليل جبران/Jihad El,The Prophet
7572,0001713191,4.17,5,P.D. Eastman,Sam And The Firefly (Beginner Books)
4538,0002310198,3.87,47,Agatha Christie,After the Funeral
4519,0002317850,3.95,19657,Agatha Christie,Sleeping Murder (Miss Marple #13)
6088,0006163831,3.98,174,Stephen R. Donaldson,The One Tree (The Second Chronicles of Thomas ...
...,...,...,...,...,...
5964,987224975X,3.97,7,Grant Morrison/Frank Quitely/Rafael De la Iglesia,JLA: Tierra 2
8371,9875661708,3.70,47,Chuck Palahniuk,Asfixia
8252,9875801445,4.32,66,Irvin D. Yalom,El Dia Que Nietzsche Lloró
5930,9879397509,3.59,116,Chuck Palahniuk/Javier Calvo,Fantasmas


In [20]:
print("jumlah ISBN: ", len(fix_book.isbn.unique()))
print("jumlah Book-Title: ", len(fix_book["title"].unique()))

jumlah ISBN:  11123
jumlah Book-Title:  10348


In [21]:
preparation = fix_book
preparation.sort_values("isbn")

,isbn,average_rating,ratings_count,authors,title
781,000100039X,4.23,184293,Kahlil Gibran/جبران خليل جبران/Jihad El,The Prophet
7572,0001713191,4.17,5,P.D. Eastman,Sam And The Firefly (Beginner Books)
4538,0002310198,3.87,47,Agatha Christie,After the Funeral
4519,0002317850,3.95,19657,Agatha Christie,Sleeping Murder (Miss Marple #13)
6088,0006163831,3.98,174,Stephen R. Donaldson,The One Tree (The Second Chronicles of Thomas ...
...,...,...,...,...,...
5964,987224975X,3.97,7,Grant Morrison/Frank Quitely/Rafael De la Iglesia,JLA: Tierra 2
8371,9875661708,3.70,47,Chuck Palahniuk,Asfixia
8252,9875801445,4.32,66,Irvin D. Yalom,El Dia Que Nietzsche Lloró
5930,9879397509,3.59,116,Chuck Palahniuk/Javier Calvo,Fantasmas


menghapus data duplikat berdasarkan isbn

In [22]:
preparation = preparation.drop_duplicates("isbn")
preparation

,isbn,average_rating,ratings_count,authors,title
781,000100039X,4.23,184293,Kahlil Gibran/جبران خليل جبران/Jihad El,The Prophet
7572,0001713191,4.17,5,P.D. Eastman,Sam And The Firefly (Beginner Books)
4538,0002310198,3.87,47,Agatha Christie,After the Funeral
4519,0002317850,3.95,19657,Agatha Christie,Sleeping Murder (Miss Marple #13)
6088,0006163831,3.98,174,Stephen R. Donaldson,The One Tree (The Second Chronicles of Thomas ...
...,...,...,...,...,...
5964,987224975X,3.97,7,Grant Morrison/Frank Quitely/Rafael De la Iglesia,JLA: Tierra 2
8371,9875661708,3.70,47,Chuck Palahniuk,Asfixia
8252,9875801445,4.32,66,Irvin D. Yalom,El Dia Que Nietzsche Lloró
5930,9879397509,3.59,116,Chuck Palahniuk/Javier Calvo,Fantasmas


## Model Development dengan Content Based Filtering

mencetak 5 sampel pada data

In [23]:
data = preparation.drop(columns=["ratings_count"])
data.sample(5)

,isbn,average_rating,authors,title
2559,0375757996,4.24,Aristotle/Richard Peter McKeon/C.D.C. Reeve,The Basic Works of Aristotle
5460,0425166821,3.82,Tom Clancy/Leon A. Edney,Carrier: A Guided Tour of an Aircraft Carrier ...
11107,8466302298,3.92,Mario Vargas Llosa,La tía Julia y el escribidor
10544,0316955124,3.93,Herman Wouk,Don't Stop the Carnival
667,0517147688,4.55,Jane Austen/Hugh Thomson,The Complete Novels


melakukan inisialisasi

In [24]:
# Inisialisasi TfidfVectorizer
tf = TfidfVectorizer()

# Melakukan perhitungan idf pada data authors
tf.fit(data["authors"]) 
 
# Mapping array dari fitur index integer ke fitur nama
tf.get_feature_names() 

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['19',
 'aaron',
 'ab',
 'abadia',
 'abaire',
 'abani',
 'abbe',
 'abbey',
 'abbie',
 'abbott',
 'abdal',
 'abdul',
 'abdulin',
 'abe',
 'abel',
 'abelson',
 'aberg',
 'abigail',
 'abolqasem',
 'abraham',
 'abrahams',
 'abrams',
 'abramson',
 'abu',
 'academy',
 'acemoğlu',
 'acevedo',
 'achcar',
 'achebe',
 'acker',
 'ackerman',
 'ackroyd',
 'acosta',
 'aczel',
 'ada',
 'adair',
 'adam',
 'adamo',
 'adams',
 'adamson',
 'adcock',
 'addington',
 'addison',
 'adela',
 'adelman',
 'adelstain',
 'adi',
 'adichie',
 'adlard',
 'adler',
 'adolfo',
 'adrian',
 'adrienne',
 'adrián',
 'adèle',
 'aeon',
 'aers',
 'aeschylus',
 'aesop',
 'aftel',
 'agar',
 'agassi',
 'agassiz',
 'agatha',
 'agatston',
 'agel',
 'agnes',
 'agnese',
 'agnew',
 'agnolo',
 'agostino',
 'agovino',
 'agrippa',
 'aharoni',
 'ahern',
 'ai',
 'aickman',
 'aida',
 'aidan',
 'aigner',
 'aiken',
 'aikins',
 'aimee',
 'aimé',
 'aine',
 'aitchison',
 'aitken',
 'ajhar',
 'akcakaya',
 'akers',
 'akert',
 'akhil',
 'akib',
 'a

melakukan fungsi fit kemudian ditrasformasikan ke dalam bentuk matrix

In [25]:
# Melakukan fit lalu ditransformasikan ke bentuk matrix
tfidf_matrix = tf.fit_transform(data["authors"])

# Melihat ukuran matrix tfidf
tfidf_matrix.shape 

(11123, 8472)

Mengubah vektor tf-idf dalam bentuk matriks

In [26]:
# Mengubah vektor tf-idf dalam bentuk matriks dengan fungsi todense()
tfidf_matrix.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

mebuat dataframe

In [27]:
# Membuat dataframe untuk melihat tf-idf matrix
 
pd.DataFrame(
    tfidf_matrix.todense(), 
    columns=tf.get_feature_names(),
    index=data.title
).sample(22, axis=1).sample(10, axis=0)

,tamara,little,ariosto,kreeft,stross,grimes,riser,orloff,raph,michalski,...,sewall,wiberg,woolf,mitchard,upledger,hessen,thomése,arrowsmith,fillbach,kosiński
title,,,,,,,,,,,,,,,,,,,,,
The Denial of Death,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
The Annotated Waste Land with Eliot's Contemporary Prose,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Great Speeches by African Americans: Frederick Douglass Sojourner Truth Dr. Martin Luther King Jr. Barack Obama and Others,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bill Bryson's African Diary,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Spares,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Blame It on Paris,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Pretty Face Vol. 1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Object Thinking,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Big Dead Place: Inside the Strange and Menacing World of Antarctica,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


menghitung kesamaan pada fitur

In [28]:
# Menghitung cosine similarity pada matrix tf-idf
cosine_sim = cosine_similarity(tfidf_matrix) 
cosine_sim

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

membuat dataframe dari variabel cosine_sim dengan baris dan kolom berupa judul buku

In [29]:
# Membuat dataframe dari variabel cosine_sim dengan baris dan kolom berupa judul buku
cosine_sim_df = pd.DataFrame(cosine_sim, index=data['title'], columns=data['title'])
print('Shape:', cosine_sim_df.shape)
 
# Melihat similarity matrix pada setiap judul buku
cosine_sim_df.sample(5, axis=1).sample(10, axis=0)

Shape: (11123, 11123)


title,The Paris Review Interviews I: 16 Celebrated Interviews,From Babylon to Timbuktu: A History of the Ancient Black Races Including the Black Hebrews,Narcissus in Chains (Anita Blake Vampire Hunter #10),Scripta Minora: Hiero/Agesilaus/Constitution of the Lacedaemonians/Ways & Means/Cavalry Commander/Art of Horsemanship/On Hunting/Constitution of the Athenians,Pearl and Wagner: Three Secrets
title,,,,,
My Wicked Wicked Ways,0.0,0.0,0.0,0.0,0.0
Enchantment,0.0,0.0,0.0,0.0,0.0
A Walk in the Woods: Rediscovering America on the Appalachian Trail,0.0,0.0,0.0,0.0,0.0
The Sons,0.0,0.0,0.0,0.0,0.0
The Man Who Loved Jane Austen (The Man Who Loved Jane Austen #1),0.0,0.0,0.0,0.0,0.0
The Gilgamesh Epic and Old Testament Parallels,0.0,0.0,0.0,0.0,0.0
Identity Crisis,0.0,0.0,0.0,0.0,0.0
Fevre Dream,0.0,0.0,0.0,0.0,0.0
The White Wolf's Son: The Albino Underground (Elric & Oona Von Bek #3),0.0,0.0,0.0,0.0,0.0


membuat fungsi book_recommendations

In [30]:
def book_recommendations(judul_buku, similarity_data=cosine_sim_df, items=data[['title', 'authors']], k=5):

    # Mengambil data dengan menggunakan argpartition untuk melakukan partisi secara tidak langsung sepanjang sumbu yang diberikan    
    # Dataframe diubah menjadi numpy
    # Range(start, stop, step)
    index = similarity_data.loc[:,judul_buku].to_numpy().argpartition(
        range(-1, -k, -1))
    
    # Mengambil data dengan similarity terbesar dari index yang ada
    closest = similarity_data.columns[index[-1:-(k+2):-1]]
    
    # Drop nama_resto agar nama resto yang dicari tidak muncul dalam daftar rekomendasi
    closest = closest.drop(judul_buku, errors='ignore')
 
    return pd.DataFrame(closest).merge(items).head(k)

In [31]:
data[data.title.eq("See How She Dies")]

,isbn,average_rating,authors,title
6490,0821776053,3.9,Lisa Jackson,See How She Dies


mendapatkan hasil rekomendasi

In [32]:
# Mendapatkan rekomendasi judul buku yang mirip dengan See How She Dies
book_recommendations("See How She Dies")

,title,authors
0,Galapagos: A Natural History,Michael H. Jackson
1,ER Vets: Life in an Animal Emergency Room,Donna M. Jackson
2,Wallace Stegner: His Life and Work,Jackson J. Benson
3,Hide (Detective D.D. Warren #2),Lisa Gardner
4,The Third Victim (Quincy & Rainie #2),Lisa Gardner
